In [ ]:
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

SERVER = os.getenv("SERVER")
DATABASE = os.getenv("DATABASE")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

# Define the connection string for SQL Server (Azure)
CONN_STR = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USERNAME};"
    f"PWD={PASSWORD}"
)

In [ ]:
def get_data() -> pd.DataFrame:
    """
    Get initial data from the Azure SQL Database.

    Returns:
        pd.DataFrame: DataFrame containing the data from the database.
    """

    try:
        conn = pyodbc.connect(CONN_STR)
        print("Connected to the Azure SQL Database successfully!")

    except pyodbc.Error as e:
        print(f"Error connecting to the database: {e}")

    query = "SELECT * FROM SalesLT.customerFROM"

    try:
        df = pd.read_sql(query, conn)
        print("Data retrieved successfully!")
        conn.close()
    except Exception as e:
        print(f"Error reading data: {e}")
        return pd.DataFrame()

    return df


df: pd.DataFrame = get_data()

### Prueba ###
data = df.head(5)
data2 = data.copy()

df.head()

In [ ]:
def prepare_data(df_: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the data for prediction.

    Args:
        df (pd.DataFrame): The input DataFrame containing the data.

    Returns:
        pd.DataFrame: The prepared DataFrame with the necessary features.
    """

    df = df_.copy()

    df = df.sort_values(by="ModifiedDate")

    df = df[["FirstName", "ModifiedDate"]]
    df["FirstNameLen"] = df["FirstName"].str.len()
    df["ModifiedDate"] = (
        pd.to_datetime(df["ModifiedDate"]) - pd.to_datetime(df["ModifiedDate"]).min()  # type: ignore
    )
    df["ModifiedDate"] = df["ModifiedDate"].dt.days

    return df[["FirstNameLen", "ModifiedDate"]].reset_index(drop=True)  # type: ignore


In [ ]:
data = prepare_data(df)
data_dict = data.to_dict(orient="list")

In [ ]:
import json

# Formateado para la API:
data_json = json.dumps({"data": [data_dict]})

URI = os.getenv("URI")

In [ ]:
import requests

headers = {"Content-Type": "application/json"}

response = requests.post(
    URI,  # type: ignore
    data=data_json,
    headers=headers,
)

if response.status_code == 200:
    result = json.loads(response.json())
    print("Predicciones del modelo:", result)
    data["Exited"] = result
    print("Variable objetivo real:", list(data2["ModifiedDate"]))
    # Print la precisión del modelo usando el porcentaje de aciertos
    aciertos = 0
    for i in range(len(data)):
        if data["Exited"][i] == list(data2["Bankrupt?"])[i]:
            aciertos += 1
    precision = aciertos / len(data) * 100
    print(f"Precisión del modelo: {precision:.2f}%")
else:
    print(f"Error: {response.text}")